In [88]:
import numpy as np
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep

In [50]:
# vytvoříme si slovník s kraji a jejich labely, které jim web přiřadil
link = f'https://www.in-pocasi.cz/archiv/archiv.php?historie=2021-10-08&region=1'
raw = requests.get(link).content
soup = BeautifulSoup(raw, 'html.parser')
labels = []
names = []
for region in soup.find("select", {"name": "archiv_kraj"}).findAll("option"):
    labels.append(region["value"])

    if region.text == "Praha":
        name = "Praha"
    elif region.text == "Vysočina":
        name = "Vysočina"
    else: name = region.text + " kraj"
    names.append(name)
regions = dict(zip(labels, names))

In [115]:
# web obsahuje dva typy meteor. stanic a každý má trochu jiné měřené hodnoty, proto ze začátku vytvoříme 2 datasety
list_klimaticke_stanice = []
list_soukrome_stanice = []
date = "2021-10-08"
for region_label in list(regions.keys()): # projdeme všechny kraje

    link = f'https://www.in-pocasi.cz/archiv/archiv.php?historie={date}&region={region_label}'
    raw = requests.get(link).content
    soup = BeautifulSoup(raw, 'html.parser') # uložíme zdrojový kód
    soup_tables = soup.find('div',{'class':'typography'}).findAll("tbody") # tabulky s daty

    for type, table in enumerate(soup_tables):
        soup_rows = table.findAll("tr")

        for row in soup_rows:
            data = [] # list na data z konkrétní stanice
            
            for element in row.findAll("td"): # přiřazujeme všechny hodnoty
                data.append(element.text)
            data.append(regions[region_label]) # přiřadíme kraj
            data.append(date) #přiřadíme datum
            data.append(row.findAll("td")[0].find("a")["href"]) # uložíme si odkaz na meteor. stanici

            if type == 0: # přidáme do příslušného listu
                list_klimaticke_stanice.append(data)
            else: list_soukrome_stanice.append(data)

# vytvoříme datasety s neupravenými daty (pouze stringy)
cols_klima = ["Stanice", "Maximální teplota (°C)", "Minimální teplota (°C)", "Náraz větru (km/h)", "Srážky (mm)", "Sněhová pokrývka (cm)", "Sluneční svit (hod)", "Kraj", "Datum", "Odkaz"]
cols_soukr = ["Stanice", "Maximální teplota (°C)", "Minimální teplota (°C)", "Náraz větru (km/h)", "Srážky (mm)", "Nejvyšší tlak (hPa)", "Nejvyšší vlhkost (%)", "Kraj", "Datum", "Odkaz"]
raw_df_klima = pd.DataFrame(list_klimaticke_stanice, columns=cols_klima)
raw_df_soukr = pd.DataFrame(list_soukrome_stanice, columns=cols_soukr)

# převedeme data na čísla
def substitute(string):
    parts = string.split(" ")
    if len(parts) == 1:
        return np.nan
    elif len(parts) == 2:
        try:
            return float(parts[0])
        except ValueError as err:
            print("Inappropriate format: ", err) 
    else: raise ValueError("Inappropriate format!")

def modify_values(init_df):
    df = init_df.copy()
    for col in df.columns[1:7]:
        df[col] = df[col].apply(substitute)
    return df

df_klima = modify_values(raw_df_klima)
df_soukr = modify_values(raw_df_soukr)

In [116]:
df_klima


,Stanice,Maximální teplota (°C),Minimální teplota (°C),Náraz větru (km/h),Srážky (mm),Sněhová pokrývka (cm),Sluneční svit (hod),Kraj,Datum,Odkaz
0,České Budějovice - Rožnov,15.5,4.8,NaN,0.0,NaN,5.0,Jihočeský kraj,2021-10-08,https://www.in-pocasi.cz/archiv/ceske_budejovice/
1,Churáňov,7.8,4.2,32.4,1.6,NaN,1.7,Jihočeský kraj,2021-10-08,https://www.in-pocasi.cz/archiv/churanov/
2,Kocelovice,14.1,5.3,39.6,0.0,NaN,5.4,Jihočeský kraj,2021-10-08,https://www.in-pocasi.cz/archiv/kocelovice/
3,Temelín,13.6,5.3,NaN,0.0,NaN,3.7,Jihočeský kraj,2021-10-08,https://www.in-pocasi.cz/archiv/temelin/
4,Brno - Tuřany,16.0,6.3,43.2,0.0,NaN,10.0,Jihomoravský kraj,2021-10-08,https://www.in-pocasi.cz/archiv/brno/
5,Kuchařovice,16.2,4.6,NaN,0.0,NaN,7.7,Jihomoravský kraj,2021-10-08,https://www.in-pocasi.cz/archiv/kucharovice/
6,Cheb,15.9,4.3,NaN,0.0,NaN,6.9,Karlovarský kraj,2021-10-08,https://www.in-pocasi.cz/archiv/cheb/
7,Karlovy Vary - Olšová Vrata,14.0,4.0,NaN,0.0,NaN,6.5,Karlovarský kraj,2021-10-08,https://www.in-pocasi.cz/archiv/karlovy_vary/
8,Pec pod Snežkou,13.0,0.0,NaN,0.0,NaN,7.3,Královéhradecký kraj,2021-10-08,https://www.in-pocasi.cz/archiv/pec_pod_snezkou/
9,Polom,12.1,3.5,57.6,0.0,NaN,9.5,Královéhradecký kraj,2021-10-08,https://www.in-pocasi.cz/archiv/polom/


In [107]:
df_klima


,Stanice,Maximální teplota (°C),Minimální teplota (°C),Náraz větru (km/h),Srážky (mm),Sněhová pokrývka (cm),Sluneční svit (hod),Kraj,Odkaz
0,České Budějovice - Rožnov,15.5,4.8,NaN,0.0,NaN,5.0,Jihočeský kraj,https://www.in-pocasi.cz/archiv/ceske_budejovice/
1,Churáňov,7.8,4.2,32.4,1.6,NaN,1.7,Jihočeský kraj,https://www.in-pocasi.cz/archiv/churanov/
2,Kocelovice,14.1,5.3,39.6,0.0,NaN,5.4,Jihočeský kraj,https://www.in-pocasi.cz/archiv/kocelovice/
3,Temelín,13.6,5.3,NaN,0.0,NaN,3.7,Jihočeský kraj,https://www.in-pocasi.cz/archiv/temelin/
4,Brno - Tuřany,16.0,6.3,43.2,0.0,NaN,10.0,Jihomoravský kraj,https://www.in-pocasi.cz/archiv/brno/
5,Kuchařovice,16.2,4.6,NaN,0.0,NaN,7.7,Jihomoravský kraj,https://www.in-pocasi.cz/archiv/kucharovice/
6,Cheb,15.9,4.3,NaN,0.0,NaN,6.9,Karlovarský kraj,https://www.in-pocasi.cz/archiv/cheb/
7,Karlovy Vary - Olšová Vrata,14.0,4.0,NaN,0.0,NaN,6.5,Karlovarský kraj,https://www.in-pocasi.cz/archiv/karlovy_vary/
8,Pec pod Snežkou,13.0,0.0,NaN,0.0,NaN,7.3,Královéhradecký kraj,https://www.in-pocasi.cz/archiv/pec_pod_snezkou/
9,Polom,12.1,3.5,57.6,0.0,NaN,9.5,Královéhradecký kraj,https://www.in-pocasi.cz/archiv/polom/
